# Human detection - Improve resolution + YOLOv4-mish
**Warning - Notebook runned on colab.research platform**

Also, don't forget to activate GPU

# Prerequies - download & install

### **human-detection** package

In [1]:
!git clone https://github.com/artzet-s/human-detection
%cd human-detection
%run setup.py install
# WARNING - You must restart environement after
%cd ..

Cloning into 'human-detection'...
remote: Enumerating objects: 74, done.
remote: Counting objects: 100% (74/74), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 74 (delta 34), reused 63 (delta 23), pack-reused 0
Unpacking objects: 100% (74/74), done.
/content/human-detection
running install
running bdist_egg
running egg_info
creating human_detection.egg-info
writing human_detection.egg-info/PKG-INFO
writing dependency_links to human_detection.egg-info/dependency_links.txt
writing top-level names to human_detection.egg-info/top_level.txt
writing manifest file 'human_detection.egg-info/SOURCES.txt'
writing manifest file 'human_detection.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build
creating build/lib
creating build/lib/human_detection
copying human_detection/json_tool.py -> build/lib/human_detection
copying human_detection/transform_labels.py -> build/lib/human_detection
copying human

zip_safe flag not set; analyzing archive contents...
human_detection.__pycache__.__init__.cpython-36: module references __path__


creating dist
creating 'dist/human_detection-0.0.1-py3.6.egg' and adding 'build/bdist.linux-x86_64/egg' to it
removing 'build/bdist.linux-x86_64/egg' (and everything under it)
Processing human_detection-0.0.1-py3.6.egg
creating /usr/local/lib/python3.6/dist-packages/human_detection-0.0.1-py3.6.egg
Extracting human_detection-0.0.1-py3.6.egg to /usr/local/lib/python3.6/dist-packages
Adding human-detection 0.0.1 to easy-install.pth file

Installed /usr/local/lib/python3.6/dist-packages/human_detection-0.0.1-py3.6.egg
Processing dependencies for human-detection==0.0.1
Finished processing dependencies for human-detection==0.0.1
/content


# **Restart your environment ! In order to have human_detection package** (CTRL + M)

### **Darknet** github (YOLOv4)

In [1]:
# clone darknet repo
!git clone https://github.com/AlexeyAB/darknet

# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
# IF YOU DON'T HAVE GPU ACCESS COMMENT THE 3 FOLLOWING LINE
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile

# verify CUDA
!/usr/local/cuda/bin/nvcc --version

# make darknet (builds darknet so that you can then use the darknet executable file to run or train object detectors)
!make

Cloning into 'darknet'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 14639 (delta 4), reused 8 (delta 3), pack-reused 14621
Receiving objects: 100% (14639/14639), 13.25 MiB | 20.82 MiB/s, done.
Resolving deltas: 100% (9959/9959), done.
/content/darknet
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -DCUDNN_HALF -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -DCUDNN_HALF -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_d

# Prepare the data

## 1. Download youtube video from url with youtube-dl

After installed youtube-dl following their documentation (https://github.com/ytdl-org/youtube-dl) use it to download the best quality video :

In [2]:
video_url = "https://youtu.be/h4s0llOpKrU"
video_filename = "/content/input_video.mp4"

!pip install --upgrade youtube-dl
!youtube-dl -F $video_url
!youtube-dl -f 137 $video_url  -o $video_filename 

     |████████████████████████████████| 1.9MB 5.7MB/s 
[youtube] h4s0llOpKrU: Downloading webpage
[info] Available formats for h4s0llOpKrU:
format code  extension  resolution note
249          webm       audio only tiny   53k , opus @ 50k (48000Hz), 282.96KiB
250          webm       audio only tiny   71k , opus @ 70k (48000Hz), 374.94KiB
140          m4a        audio only tiny  130k , m4a_dash container, mp4a.40.2@128k (44100Hz), 713.10KiB
251          webm       audio only tiny  140k , opus @160k (48000Hz), 738.09KiB
394          mp4        256x144    144p   71k , av01.0.00M.08, 25fps, video only, 363.07KiB
278          webm       256x144    144p   92k , webm container, vp9, 25fps, video only, 497.53KiB
160          mp4        256x144    144p  110k , avc1.4d400c, 25fps, video only, 429.41KiB
395          mp4        426x240    240p  150k , av01.0.00M.08, 25fps, video only, 730.89KiB
242          webm       426x240    240p  224k , vp9, 25fps, video only, 972.57KiB
133          mp4      

## 2. Extract video frames


In [3]:
import human_detection
from human_detection.video_frames import video_to_frames, \
                                         write_txt_list_of_filename

frames_dir = "/content/input_frames"
frames_txt = "/content/input_frames.txt"

video_to_frames(video_filename, frames_dir, False)
write_txt_list_of_filename(frames_dir, frames_txt)

KeyboardInterrupt: ignored

## 3. Load models weights

According : 
* https://github.com/AlexeyAB/darknet#pre-trained-models
* https://github.com/AlexeyAB/darknet/wiki/YOLOv4-model-zoo

In [ ]:
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v4_pre/yolov4x-mish.weights

# Run models

# Utilities


In [ ]:
import os
import glob
import cv2
import json

def split_frames(frame_dir, 
                 output_dir,
                 output_txt):

  if not os.path.exists(output_dir):
    os.mkdir(output_dir)

  square_size = 640
  number_split = (6, 4)

  data = list()
  filenames = glob.glob(os.path.join(frame_dir, "*.jpg"))

  filenames_txt = list()
  for i in range(len(filenames)):

    filename = os.path.join(frame_dir, "{}.jpg".format(i))
    
    img = cv2.imread(filename)
    height, width, layers = img.shape

    wr =(width - square_size) / 5
    hr =(height - square_size) / 3

    print(filename)
    
    for j in range(6):
      for k in range(4):
        start_w = int(wr * j)
        end_w = start_w + square_size
        start_h = int(hr * k)
        end_h = start_h + square_size

        im = img[start_h:end_h, start_w: end_w]
        filename_output = os.path.join(output_dir, "{}-{}-{}.jpg".format(i, start_w, start_h))
        cv2.imwrite(filename_output, im)
        filenames_txt.append(filename_output)
    
  file = open(output_txt, 'w')
  file.write('\n'.join(filenames_txt))
  file.close()

    

split_frames(frame_dir="/content/input_frames", 
             output_dir="/content/splitted_frames",
             output_txt="/content/splitted.txt")

In [ ]:
model = "yolov4x-mish"

frames_txt = "/content/splitted.txt"
cfg_model = "cfg/{}.cfg".format(model)
weights_model = "{}.weights".format(model)
result = "/content/{}_result.json".format(model)
pred_frames_dir = "/content/{}_frames".format(model)
pred_video = "/content/{}_video.mp4".format(model)

!./darknet detector test \
  "cfg/coco.data" \
  $cfg_model \
  $weights_model \
  -dont_show \
  -out $result \
  < $frames_txt


In [ ]:
import collections

def merge_splitted(main_result, grid_result, output_json):
  
  img_height, img_width = (1080, 1920)
  square_size = 640

  with open(zoom_result) as f:
    grid_data = json.load(f)

  with open(main_result) as f:
    main_data = json.load(f)
  
  merged = dict()
  for d in main_data:
    name = os.path.basename(d["filename"])[:-4]
    md = dict()
    md['filename'] = "/content/input_frames/{}.jpg".format(name)
    md['main_objects'] = d['objects']
    merged[name] = md

  
  for d in grid_data:

    name, x, y = os.path.basename(d["filename"])[:-4].split('-')
    md = merged[name]

    if name in merged:  
    else:
      md = dict()
      md['filename'] = "/content/input_frames/{}.jpg".format(name)
      md['objects'] = list()
      merged[name] = md


    for obj in d["objects"]:
        rc = obj["relative_coordinates"]

        rc["center_x"] = (int(x) + rc["center_x"] * square_size) / img_width
        rc["center_y"] = (int(y) + rc["center_y"] * square_size) / img_height
        rc["width"] = rc["width"] * square_size / img_width
        rc["height"] = rc["height"] * square_size / img_height

        md['objects'].append(obj)
        
  with open(output_json, 'w') as outfile:
    json.dump(list(merged.values()), outfile, indent=4)

      
merge_splitted("/content/yolov4x-mish_result.json",
               "/content/yolov4x-mish_splitted_result.json",
               "/content/result.json")

In [ ]:
from human_detection.video_frames import frames_to_video
from human_detection.draw_box import draw_person_box_on_frames

pred_frames_dir = "/content/merged_frames"
pred_video = "/content/merged_video.mp4"


draw_person_box_on_frames("/content/result.json", 
                          pred_frames_dir,
                          confidence=0.25)

frames_to_video(pred_frames_dir, 
                video_filename=pred_video, 
                fps=25)